In [1]:
# Data processing
import pandas as pd

test_prompt_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/test_prompt_data_task1_simple.csv")
print(test_prompt_data.shape)
test_prompt_data.head()

(2312, 3)


,cell_id,prompt,answer
0,ACH-000006,Think step by step and decide in a single word...,Sensitive
1,ACH-000006,Think step by step and decide in a single word...,Resistant
2,ACH-000006,Think step by step and decide in a single word...,Resistant
3,ACH-000006,Think step by step and decide in a single word...,Sensitive
4,ACH-000006,Think step by step and decide in a single word...,Sensitive


In [2]:
test_prompt_data.iloc[219:,:] # N= 219

,cell_id,prompt,answer
219,ACH-000139,Think step by step and decide in a single word...,Sensitive
220,ACH-000139,Think step by step and decide in a single word...,Sensitive
221,ACH-000139,Think step by step and decide in a single word...,Resistant
222,ACH-000139,Think step by step and decide in a single word...,Resistant
223,ACH-000139,Think step by step and decide in a single word...,Resistant
...,...,...,...
2307,ACH-001654,Think step by step and decide in a single word...,Sensitive
2308,ACH-001654,Think step by step and decide in a single word...,Sensitive
2309,ACH-001654,Think step by step and decide in a single word...,Sensitive
2310,ACH-001654,Think step by step and decide in a single word...,Resistant


In [3]:
test_samples = test_prompt_data.iloc[:200,:].copy()
test_samples['output70'] = [None,]*200
# test_samples = pd.read_csv('./data/sensitivity/test_fewshot_simple.csv')

In [4]:
test_samples.head()

,cell_id,prompt,answer,output70
0,ACH-000006,Think step by step and decide in a single word...,Sensitive,None
1,ACH-000006,Think step by step and decide in a single word...,Resistant,None
2,ACH-000006,Think step by step and decide in a single word...,Resistant,None
3,ACH-000006,Think step by step and decide in a single word...,Sensitive,None
4,ACH-000006,Think step by step and decide in a single word...,Sensitive,None


In [5]:
fewshot_set = test_prompt_data.iloc[219:,:] #1. not appear in train data; 2. not to inference (test data top 200 -> inf)
fewshot_set.shape

(2093, 3)

In [6]:
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
model_dir = "/data/yingfei/models/llm/medAlpaca/medalpaca-7b"

model = LlamaForCausalLM.from_pretrained(model_dir, device_map = 'auto')#.to(DEVICE)
tokenizer = LlamaTokenizer.from_pretrained(model_dir)#.to(model.device)

# from inferer import Inferer
# medalpaca = Inferer(
#     model_name="medalpaca/medalapca-7b", 
#     prompt_template="path/to/your/prompt_template.json",
#     base_model=None,  # Optional, specify if using LoRA or another base model
#     model_max_length=512,  # Optional, can be adjusted based on your input needs
#     load_in_8bit=False,  # Optional, set to True if using a quantized model
#     torch_dtype=torch.float16,  # Optional, can be adjusted based on your precision needs
#     peft=False  # Optional, set to True if the model was trained with PEFT or in 8bit
# )

/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
import numpy as np

def get_few_shot(fewshot_set, num=5):
    fewshot = ''
    for i in range(num):#len(fewshot_set)):
        idx = np.random.randint(0,len(fewshot_set))
        answer = fewshot_set.iloc[idx].answer
        question = fewshot_set.iloc[idx].prompt.split("[Reasoning].")[1].replace('?',answer)#.replace("\n", "")
        fewshot += f'\n\nExample {i}:\n'
        fewshot += question#.strip()
    
    return fewshot.strip()

In [8]:
for idx in range(200):
    row = test_samples.iloc[idx,:]
    if not pd.isna(row.output70): continue
    print(f'PROCESSING CELLINE: {row.cell_id}   ', idx)
    prompt = row.prompt

    pipeline = transformers.pipeline("text-generation", model=model,
                                    tokenizer=tokenizer, temperature = 0.0,
                                    torch_dtype=torch.float16, device_map="auto",
                                    return_full_text=False)
    instruction = "Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]"
    prompt = prompt.split("[Reasoning].")[1]
    #fewshot = get_few_shot(fewshot_set, num=5)
    #prompt_content = f"<s>[INST] <<SYS>>{instruction}<</SYS>> <<EXAMPLE>>{fewshot}<</EXAMPLE>> {prompt}[/INST]"
    prompt_content = f"<s>[INST] <<SYS>>{instruction}<</SYS>> {prompt}[/INST]"
    
    print(prompt_content)
    # Run prompt and pipeline
    sequences = pipeline(prompt_content, truncation=True, max_length=20480)
    # print(result[0]['generated_text']) # Not in one word??? !!!
    output = ""
    for seq in sequences:
        new_output = seq['generated_text']
        print(new_output)
        output += new_output
    

    test_samples.loc[idx,'output70'] = output.strip()
    test_samples.to_csv('test_output.csv', index=None)
    print()
    print(f'GROUND TRUTH: {row.answer}')
    print('*'*20)

PROCESSING CELLINE: ACH-000006    0
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> 
Drug and cell line mutations: 
The drug is JW-7-24-1. The drug SMILES structure is COC1=CC(=CC(=C1)C2=CC3=C4C(=CN=C3C=C2)C=CC(=O)N4C5=CC(=C(C=C5)N6CCNCC6)C(F)(F)F)OC. Drug target is LCK. Drug target pathway is Other, kinases.
The mutations of the cell line are ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1.
Drug Sensitivity: ?[/INST]


/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Drug Resistance:?[/INST]

GROUND TRUTH: Sensitive
********************
PROCESSING CELLINE: ACH-000006    1
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> 
Drug and cell line mutations: 
The drug is NVP-TAE684. The drug SMILES structure is CC(C)S(=O)(=O)C1=CC=CC=C1NC2=NC(=NC=C2Cl)NC3=C(C=C(C=C3)N4CCC(CC4)N5CCN(CC5)C)OC. Drug target is ALK. Drug target pathway is RTK signaling.
The mutations of the cell line are ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1.
Drug Sensitivity: ?[/INST]


/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(




Drug and cell line mutations: 
The drug is erlotinib. The drug SMILES structure is C1=CC=CC=C1NC2=NC(=NC=C2Cl)NC3=C(C=C(C=C3)N4CCC(CC4)N5CCN(CC5)C)OC. Drug target is EGFR. Drug target pathway is RTK signaling.
The mutations of the cell line are EGFR, KRAS, MET, PIK3CA, PTEN, SMAD4, TP53.
Drug Sensitivity: [Sensitive]

GROUND TRUTH: Resistant
********************
PROCESSING CELLINE: ACH-000006    2
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> 
Drug and cell line mutations: 
The drug is KIN001-244. The drug SMILES structure is C1=CC=C(C=C1)[C@H](COC2=CC3=C(C=C2)NC(=O)N3)NC(=O)C4=CC=CN(C4=O)CC5=CC(=C(C=C5)F)F. Drug target is PDK1. Drug target pathway is Other, kinases.
The mutations of the cell line are ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1.
Drug Sensitivity: ?[/INST]


/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Drug Resistance:?[/INST]

GROUND TRUTH: Resistant
********************
PROCESSING CELLINE: ACH-000006    3
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> 
Drug and cell line mutations: 
The drug is LINSITINIB. The drug SMILES structure is CC1(CC(C1)C2=NC(=C3N2C=CN=C3N)C4=CC5=C(C=C4)C=CC(=N5)C6=CC=CC=C6)O. Drug target is IGF1R. Drug target pathway is IGF1R signaling.
The mutations of the cell line are ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1.
Drug Sensitivity: ?[/INST]


/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df_result = pd.read_csv('test_output.csv')
df_result.head()

In [ ]:
pred_lst = []
for x in df_result.output70:
    pre = x.find('Resistant')
    if pre != -1:
        pred_lst.append('Resistant')
    else:
        pred_lst.append('Sensitive')

In [ ]:
pred_lst[:5]

In [ ]:
corr = [1 for idx, x in enumerate(df_result.answer) if x == pred_lst[idx]]

In [ ]:
sum(corr)/len(df_result)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(df_result.answer.tolist(), pred_lst, labels=["Resistant", "Sensitive"])

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Resistant", "Sensitive"])

In [ ]:
disp.plot()